## Do I have the right to complain about Dutch trains in a demo?
As a student, I had to take the train from Breda to Eindhoven and back multiple times a week.
I did experience a lot of delays, but can we visualize this?

I collected all data from 2011-2023 from rijdendetreinen.nl into an SQLite database which we can also access from DataFrame!

In [1]:
// Adding support for the sqlite jdbc driver before importing DataFrame
USE { dependencies("org.xerial:sqlite-jdbc:3.45.1.0") }

In [2]:
%use dataframe(v=0.13.1), kandy

Making a connection to the database and then reading the right table.

In [3]:
val connection = DatabaseConfiguration(url = "jdbc:sqlite:data/disruptions/disruptions-history.sqlite")

In [4]:
val df1 = DataFrame.readSqlTable(
    dbConfig = connection,
    tableName = "disruptions",
    limit = Int.MAX_VALUE,
)

df1

rdt_id,ns_lines,rdt_lines,rdt_lines_id,rdt_station_names,rdt_station_codes,cause_nl,cause_en,statistical_cause_nl,statistical_cause_en,cause_group,start_time,end_time,duration_minutes
40500,Den Haag-Rotterdam; Leiden-Rotterdam,Den Haag HS - Rotterdam Centraal,11,"Delft,Delft Campus,Den Haag HS,Den Ha...","DT, DTCP, GV, GVMW, RSW",brandmelding,fire alarm,brandmelding,fire alarm,external,2022-01-01 05:45:33,2022-01-01 06:34:58,49
40501,Amsterdam-Utrecht; Utrecht-Eindhoven,"'s-Hertogenbosch - Utrecht Centraal, ...","136,151","Abcoude,Amsterdam Amstel,Amsterdam Bi...","AC, ASA, ASB, ASD, ASDM, ASHD, BKL, C...",herstelwerkzaamheden,repair works,herstelwerkzaamheden,repair works,engineering work,2022-01-01 06:23:54,2022-01-01 13:59:14,455
40502,Schiphol-Rotterdam (HSL),Rotterdam Centraal - Schiphol Airport...,24,"Rotterdam Centraal,Schiphol Airport","RTD, SHL",brandmelding,fire alarm,brandmelding,fire alarm,external,2022-01-01 06:33:40,2022-01-01 06:34:27,1
40503,Eindhoven-Venlo,Eindhoven - Venlo,65,"Blerick,Deurne,Horst-Sevenum,Venlo","BR, DN, HRT, VL",aanrijding,collision,aanrijding,collision,accidents,2022-01-01 07:31:39,2022-01-01 11:26:38,235
40504,Alkmaar-Den Helder; Alkmaar-Hoorn,"Alkmaar - Den Helder, Alkmaar - Hoorn","162,163","Alkmaar Noord,Heerhugowaard","AMRN, HWD",aanrijding,collision,aanrijding,collision,accidents,2022-01-01 07:32:32,2022-01-01 07:42:25,10
40505,Schiphol Airport,"Amersfoort - Schiphol Airport, Amster...","22,24,32,137,144,148,165",Schiphol Airport,SHL,defecte trein,broken down train,defecte trein,broken down train,rolling stock,2022-01-01 07:34:49,2022-01-01 07:41:13,6
40506,Amsterdam-Schiphol-Rotterdam (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",gestrande trein,stranded train,gestrande trein,stranded train,rolling stock,2022-01-01 12:33:22,2022-01-01 12:44:05,11
40507,Sittard-Heerlen,Heerlen - Sittard,77,"Geleen Oost,Hoensbroek,Heerlen,Nuth,S...","GLN, HB, HRL, NH, SBK, SN, STD",herstelwerkzaamheden,repair works,herstelwerkzaamheden,repair works,engineering work,2022-01-01 16:06:23,2022-01-01 16:46:33,40
40508,Utrecht-Hilversum-Weesp; Hoofddorp-Sc...,"Amersfoort - Amsterdam Centraal, Amer...","135,148","Naarden-Bussum,Weesp","NDB, WP",herstelwerkzaamheden,repair works,herstelwerkzaamheden,repair works,engineering work,2022-01-01 16:17:56,2022-01-01 17:18:47,61
40509,Zwolle-Groningen,Groningen - Zwolle,146,"Assen,Groningen Europapark,Groningen,...","ASN, GERP, GN, HRN",aanrijding,collision,aanrijding,collision,accidents,2022-01-01 18:40:47,2022-01-01 22:55:38,255


The table has 50k rows, which for DataFrame is not too large, but if your table has millions of rows, it may be beneficial to only load a given query into the DF. DataFrame runs in memory to be as flexible as possible and to infer the type-safe accessors.

Let's try this with weather related disruptions and see in which months most weather related disruptions occur.

In [28]:
val weatherDisruptions = DataFrame.readSqlQuery(
    dbConfig = connection,
    sqlQuery = """SELECT * FROM disruptions WHERE cause_group="weather"""",
    limit = Int.MAX_VALUE,
).parse().renameToCamelCase()

weatherDisruptions

rdtId,nsLines,rdtLines,rdtLinesId,rdtStationNames,rdtStationCodes,causeNl,causeEn,statisticalCauseNl,statisticalCauseEn,causeGroup,startTime,endTime,durationMinutes
40603,Eindhoven-Venlo,Eindhoven - Venlo,65,"Blerick,Deurne,Eindhoven Centraal,Hel...","BR, DN, EHV, HM, HMBH, HMBV, HMH, HRT...",rijp aan de bovenleiding,overhead wires covered with frost,rijp aan de bovenleiding,overhead wires covered with frost,weather,2022-01-10T07:17:03,2022-01-10T07:38:54,22
40654,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130,"Eygelshoven Markt,Heerlen,Heerlen De ...","EGHM, HRL, HRLK, HZ, LG",rijp aan de bovenleiding,overhead wires covered with frost,rijp aan de bovenleiding,overhead wires covered with frost,weather,2022-01-11T23:45:34,2022-01-13T10:26:24,2081
40681,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130,"Eygelshoven Markt,Heerlen,Heerlen De ...","EGHM, HRL, HRLK, HZ, LG",de weersomstandigheden,weather circumstances,de weersomstandigheden,weather circumstances,weather,2022-01-14T03:06,2022-01-14T10:29:38,444
40883,Amsterdam-Zaandam-Alkmaar,Alkmaar - Amsterdam Centraal,161,"Alkmaar,Castricum,Heiloo,Krommenie-As...","AMR, CAS, HLO, KMA, UTG, WM, ZD, KZ, ZZS",de weersomstandigheden,weather circumstances,de weersomstandigheden,weather circumstances,weather,2022-01-31T04:20:18,2022-01-31T15:12:22,652
40923,Alphen a/d Rijn-Gouda,Alphen a/d Rijn - Gouda,8,"Alphen a/d Rijn,Boskoop,Gouda,Waddinx...","APN, BSK, GD, WAD, WADN, BSKS, WADT",gladde sporen,slippery railway tracks,gladde sporen,slippery railway tracks,weather,2022-02-01T06:24:09,2022-02-01T10:29:23,245
41151,Rotterdam-Breda (HSL),Breda - Rotterdam Centraal (HSL),15,"Breda,Rotterdam Centraal","BD, RTD",harde wind op de hogesnelheidslijn,strong winds on the high speed line,harde wind op de hogesnelheidslijn,strong winds on the high speed line,weather,2022-02-16T22:06:18,2022-02-17T02:30:37,264
41154,Amsterdam-Schagen,Alkmaar - Amsterdam Centraal,161,"Alkmaar,Castricum,Heiloo,Krommenie-As...","AMR, CAS, HLO, KMA, UTG, WM, ZD, KZ, ZZS",de weersomstandigheden,weather circumstances,de weersomstandigheden,weather circumstances,weather,2022-02-17T04:04:56,2022-02-17T09:52:34,348
41199,Amsterdam-Schiphol-Rotterdam (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",harde wind op de hogesnelheidslijn,strong winds on the high speed line,harde wind op de hogesnelheidslijn,strong winds on the high speed line,weather,2022-02-20T19:14:50,2022-02-20T19:29:38,15
41200,Amsterdam-Schiphol-Rotterdam (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",harde wind op de hogesnelheidslijn,strong winds on the high speed line,harde wind op de hogesnelheidslijn,strong winds on the high speed line,weather,2022-02-20T19:55:22,2022-02-20T23:32:18,217
41201,Rotterdam-Breda (HSL),Breda - Rotterdam Centraal (HSL),15,"Breda,Rotterdam Centraal","BD, RTD",harde wind op de hogesnelheidslijn,strong winds on the high speed line,harde wind op de hogesnelheidslijn,strong winds on the high speed line,weather,2022-02-20T19:58:15,2022-02-20T23:32:37,214


In [29]:
import kotlinx.datetime.Month

val month by column<Month>()

val weatherDisruptionsCounts = weatherDisruptions
    .groupBy { expr { startTime.month } into month }
    .count()
    .sortBy(month)

weatherDisruptionsCounts

month,count
JANUARY,90
FEBRUARY,88
MARCH,47
APRIL,24
MAY,44
JUNE,64
JULY,95
AUGUST,39
SEPTEMBER,20
OCTOBER,75


In [30]:
weatherDisruptionsCounts.countPlot { x(month); weight(count) }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="UlC3y7"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"x":["JANUARY","FEBRUARY","MARCH","APRIL","MAY","JUNE","JULY","AUGUST","SEPTEMBER","OCTOBER","NOVEMBER","DECEMBER"],
"countWeighted":[90.0,88.0,47.0,24.0,44.0,64.0,95.0,39.0,20.0,75.0,96.0,87.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"month",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"countWeighted"
},
"stat":"identity",
"sampling":"none",
"position":"dodge",
"geom":"bar",
"data":{
}
}]
};
 var plotContainer = document.getElementById("UlC3y7");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 JANUARY 
 
 
 
 
 
 
 
 
 FEBRUARY 
 
 
 
 
 
 
 
 
 MARCH 
 
 
 
 
 
 
 
 
 APRIL 
 
 
 
 
 
 
 
 
 MAY 
 
 
 
 
 
 
 
 
 JUNE 
 
 
 
 
 
 
 
 
 JULY 
 
 
 
 
 
 
 
 
 AUGUST 
 
 
 
 
 
 
 
 
 SEPTEMBER 
 
 
 
 
 
 
 
 
 OCTOBER 
 
 
 
 
 
 
 
 
 NOVEMBER 
 
 
 
 
 
 
 
 
 DECEMBER 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 countWeighted 
 
 
 
 
 month

But back to the entire table! Let's parse it similarly to in the other notebook, but just a bit more quickly (and unsafely).

In [31]:
import kotlin.time.Duration.Companion.minutes

/**
 * A line consists of two stations where firstStation
 * is always alphabetically first.
 */
data class Line private constructor(
    val firstStation: String,
    val secondStation: String,
) {
    companion object {
        operator fun invoke(station: String, otherStation: String): Line {
            val (a, b) = listOf(station, otherStation).sorted()
            return Line(a, b)
        }

        fun parseOrNull(rdtString: String): Line? {
            val stations = rdtString.split(" - ")
            return invoke(
                stations.getOrNull(0) ?: return null,
                stations.getOrNull(1) ?: return null,
            )
        }
    }

    override fun toString(): String = "$firstStation <-> $secondStation"
}

val allDisruptions = df1
    .parse() // parse string columns
    .renameToCamelCase()
    .remove { "nsLines" and nameEndsWith("Nl") and "causeEn" } // remove unnecessary columns
    .update { "durationMinutes"<String?>() }.where { it.isNullOrBlank() }.withZero() // imputing blank durations
    .add { // adding helper columns
        "duration" from { "durationMinutes"<String>().toInt().minutes }
        "date" from { "startTime"<LocalDateTime>().date }
    }
    .rename { all() }.into { // renaming
        it.name
            .removePrefix("rdt")
            .replaceFirstChar { it.lowercase() }
            .removeSuffix("En")
    }
    .split("lines", "linesId", "stationNames", "stationCodes").by { // splitting list-like string columns into lists
        (it as String?)
            .takeUnless { it.isNullOrBlank() }
            ?.let { it.split(",") }
            ?: emptyList()
    }.inplace()
    .convert { "linesId"<List<String>>() }.with { it.map { it.toInt() } } // converting linesId to List<Int> todo remove?
    .convert { "lines"<List<String>>() }.with { it.mapNotNull { Line.parseOrNull(it) } } // converting lines to List<Line> 
    .sortBy("startTime") // sort :)

allDisruptions

id,lines,linesId,stationNames,stationCodes,statisticalCause,causeGroup,startTime,endTime,durationMinutes,duration,date
1,[Amersfoort <-> Apeldoorn],[50],[ ],[ ],police action,external,2011-01-03T23:22:50,2011-01-04 01:35:03,132,2h 12m,2011-01-03
4,[Amersfoort <-> Ede-Wageningen],[47],[ ],[ ],police action,external,2011-01-04T00:23:13,2011-01-04 01:35:03,72,1h 12m,2011-01-04
5,[Breda <-> Roosendaal],[20],[ ],[ ],copper theft,external,2011-01-04T06:30:02,2011-01-04 10:05:01,215,3h 35m,2011-01-04
6,[Eindhoven <-> Venlo],[65],[ ],[ ],person hit by a train,accidents,2011-01-04T07:30:03,2011-01-04 11:00:08,210,3h 30m,2011-01-04
7,[Leiden Centraal <-> Utrecht Centraal],[147],[ ],[ ],broken down train,rolling stock,2011-01-04T07:30:03,2011-01-04 08:50:02,80,1h 20m,2011-01-04
8,"[Den Haag HS <-> Rotterdam Centraal, ...","[11, 16]",[ ],[ ],signalling and points failure,infrastructure,2011-01-04T10:50:05,2011-01-04 13:00:03,130,2h 10m,2011-01-04
9,[Almelo <-> Zwolle],[95],[ ],[ ],points failure,infrastructure,2011-01-04T14:20:02,2011-01-04 15:25:02,65,1h 5m,2011-01-04
10,[Amsterdam Centraal <-> Hoorn],[164],[ ],[ ],broken down train,rolling stock,2011-01-04T18:30:01,2011-01-04 20:15:02,105,1h 45m,2011-01-04
11,[Arnhem Centraal <-> Utrecht Centraal],[133],[ ],[ ],person hit by a train,accidents,2011-01-04T19:55:01,2011-01-04 22:20:01,145,2h 25m,2011-01-04
12,[Amersfoort <-> Ede-Wageningen],[47],[ ],[ ],cause yet unknown,unknown,2011-01-05T04:55:01,2011-01-06 02:25:02,1290,21h 30m,2011-01-05


Now we've got all disruptions, let's find the ones related to me.
I used to take the InterCity from Breda to Eindhoven and back. It stopped in Tilburg too.
I'll count a disruption when it happened either in Breda <-> Tilburg or Tilburg <-> Eindhoven.


In [32]:
val bredaTilburg = Line("Breda", "Tilburg")
val tilburgEindhoven = Line("Tilburg", "Eindhoven")

val relatedToMe = allDisruptions.filter {
    bredaTilburg in lines || tilburgEindhoven in lines
}

relatedToMe

id,lines,linesId,stationNames,stationCodes,statisticalCause,causeGroup,startTime,endTime,durationMinutes,duration,date
27,[Eindhoven <-> Tilburg],[152],[ ],[ ],person hit by a train,accidents,2011-01-08T02:00:03,2011-01-08 03:25:02,85,1h 25m,2011-01-08
57,[Eindhoven <-> Tilburg],[152],[ ],[ ],signal and level crossing failure,infrastructure,2011-01-13T08:30:01,2011-01-13 12:25:02,235,3h 55m,2011-01-13
68,[Eindhoven <-> Tilburg],[152],[ ],[ ],police action,external,2011-01-13T20:10:05,2011-01-13 22:15:01,125,2h 5m,2011-01-13
82,[Breda <-> Tilburg],[68],[ ],[ ],person hit by a train,accidents,2011-01-16T19:05:02,2011-01-16 21:50:03,165,2h 45m,2011-01-16
231,[Breda <-> Tilburg],[68],[ ],[ ],points failure,infrastructure,2011-02-21T06:35:01,2011-02-21 07:50:01,75,1h 15m,2011-02-21
276,[Breda <-> Tilburg],[68],[ ],[ ],person hit by a train,accidents,2011-03-03T10:50:02,2011-03-03 13:30:02,145,2h 25m,2011-03-03
328,[Breda <-> Tilburg],[68],[ ],[ ],signal and level crossing failure,infrastructure,2011-03-15T18:20:02,2011-03-15 19:50:02,75,1h 15m,2011-03-15
357,[Eindhoven <-> Tilburg],[152],[ ],[ ],broken down train,rolling stock,2011-03-24T19:30:01,2011-03-24 20:20:03,35,35m,2011-03-24
375,[Breda <-> Tilburg],[68],[ ],[ ],broken down train,rolling stock,2011-03-28T14:45:02,2011-03-28 16:20:02,80,1h 20m,2011-03-28
475,[Eindhoven <-> Tilburg],[152],[ ],[ ],collision,accidents,2011-04-16T04:14:03,2011-04-16 08:28:03,242,4h 2m,2011-04-16


We've got 11k results! That's a lot.
Let's plot them over time and see what types of disruptions were most common.

In [41]:
val relatedGrouped = relatedToMe.groupBy { // group by expression (temp) column monthAndYear 
    expr("monthAndYear") {
        LocalDate(
            year = date.year,
            month = date.month,
            dayOfMonth = 1,
        )
    }
}.aggregate {
    valueCounts { causeGroup } into "counts"
}.asGroupBy()

relatedGrouped

<iframe onload="o_resize_iframe_out_32()" style="width:100%;" class="result_container" id="iframe_out_32" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_134218150"></table>

<p class="dataframe_description">... showing only top 20 of 156 rows</p><p class="dataframe_description">GroupBy</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].p

In [44]:
relatedGrouped.plot {
    x.axis.breaks(format = "%B %Y")
    
    bars {
        x(monthAndYear)
        y("count")
        fillColor(causeGroup)
        borderLine.width = 0.0
        
        position = Position.stack()
    }

    layout.size = 1000 to 700
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="cNd5ED"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"&merged_groups":["2011-01-01","2011-01-01","2011-01-01","2011-02-01","2011-03-01","2011-03-01","2011-03-01","2011-04-01","2011-05-01","2011-05-01","2011-05-01","2011-06-01","2011-06-01","2011-06-01","2011-06-01","2011-07-01","2011-07-01","2011-07-01","2011-08-01","2011-09-01","2011-10-01","2011-10-01","2011-11-01","2011-11-01","2011-11-01","2011-11-01","2011-12-01","2012-01-01","2012-02-01","2012-02-01","2012-03-01","2012-03-01","2012-04-01","2012-04-01","2012-04-01","2012-05-01","2012-05-01","2012-05-01","2012-06-01","2012-06-01","2012-07-01","2012-07-01","2012-07-01","2012-07-01","2012-08-01","2012-08-01","2012-09-01","2012-10-01","2012-10-01","2012-11-01","2012-11-01","2012-12-01","2012-12-01","2013-01-01","2013-01-01","2013-02-01","2013-03-01","2013-03-01","2013-03-01","2013-04-01","2013-04-01","2013-04-01","2013-04-01","2013-05-01","2013-05-01","2013-05-01","2013-06-01","2013-06-01","2013-06-01","2013-07-01","2013-07-01","2013-07-01","2013-07-01","2013-08-01","2013-08-01","2013-08-01","2013-09-01","2013-09-01","2013-09-01","2013-09-01","2013-10-01","2013-10-01","2013-10-01","2013-11-01","2013-11-01","2013-12-01","2013-12-01","2014-01-01","2014-01-01","2014-01-01","2014-02-01","2014-02-01","2014-03-01","2014-03-01","2014-04-01","2014-04-01","2014-04-01","2014-04-01","2014-04-01","2014-04-01","2014-05-01","2014-05-01","2014-05-01","2014-05-01","2014-06-01","2014-06-01","2014-07-01","2014-07-01","2014-08-01","2014-08-01","2014-08-01","2014-09-01","2014-10-01","2014-10-01","2014-11-01","2014-11-01","2014-12-01","2014-12-01","2014-12-01","2014-12-01","2015-01-01","2015-01-01","2015-01-01","2015-01-01","2015-02-01","2015-02-01","2015-02-01","2015-03-01","2015-03-01","2015-03-01","2015-04-01","2015-04-01","2015-04-01","2015-04-01","2015-05-01","2015-05-01","2015-06-01","2015-06-01","2015-07-01","2015-07-01","2015-07-01","2015-07-01","2015-07-01","2015-08-01","2015-09-01","2015-09-01","2015-09-01","2015-10-01","2015-10-01","2015-10-01","2015-10-01","2015-11-01","2015-11-01","2015-11-01","2015-12-01","2015-12-01","2016-01-01","2016-01-01","2016-01-01","2016-02-01","2016-02-01","2016-03-01","2016-03-01","2016-04-01","2016-04-01","2016-04-01","2016-04-01","2016-04-01","2016-05-01","2016-05-01","2016-05-01","2016-06-01","2016-06-01","2016-06-01","2016-06-01","2016-07-01","2016-07-01","2016-07-01","2016-07-01","2016-08-01","2016-08-01","2016-08-01","2016-08-01","2016-09-01","2016-09-01","2016-10-01","2016-10-01","2016-10-01","2016-10-01","2016-11-01","2016-11-01","2016-11-01","2016-11-01","2016-11-01","2016-12-01","2016-12-01","2016-12-01","2017-01-01","2017-01-01","2017-01-01","2017-01-01","2017-02-01","2017-02-01","2017-02-01","2017-02-01","2017-03-01","2017-03-01","2017-03-01","2017-03-01","2017-03-01","2017-04-01","2017-04-01","2017-05-01","2017-05-01","2017-05-01","2017-05-01","2017-05-01","2017-06-01","2017-06-01","2017-06-01","2017-06-01","2017-07-01","2017-07-01","2017-07-01","2017-07-01","2017-08-01","2017-08-01","2017-08-01","2017-08-01","2017-09-01","2017-09-01","2017-09-01","2017-09-01","2017-10-01","2017-10-01","2017-10-01","2017-10-01","2017-11-01","2017-11-01","2017-11-01","2017-11-01","2017-12-01","2017-12-01","2017-12-01","2017-12-01","2018-01-01","2018-01-01","2018-01-01","2018-02-01","2018-02-01","2018-02-01","2018-02-01","2018-03-01","2018-03-01","2018-04-01","2018-04-01","2018-04-01","2018-04-01","2018-05-01","2018-05-01","2018-05-01","2018-05-01","2018-06-01","2018-06-01","2018-06-01","2018-06-01","2018-07-01","2018-07-01","2018-07-01","2018-07-01","2018-08-01","2018-08-01","2018-08-01","2018-08-01","2018-09-01","

## So... yes? 

## Now, will you get to complain about Dutch trains this evening too?